# Fine Tune reward model from scratch

# TODOs:

#TODO: double-check that labels are not somehow misaligned...

#TODO: check if you need to plot 

1. LoRA learns the position of the low rank adaptation matrix that is needed to finetune a model of a much higher rank

#TODO: double check model performance, generate output, maybe adjust training metrics

## 1. Imports, setup, and global variables

In [9]:
import torch
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from torch.nn import functional as F
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error, mean_absolute_error

from datasets import Dataset, DatasetDict, load_from_disk

from peft import LoraConfig, get_peft_model, PeftModel

from utils import parse_ratings

from sentence_transformers import SentenceTransformer


# from nltk.tokenize import sent_tokenize

# load the relevant devices available on the server
os.environ["CUDA_VISIBLE_DEVICES"] = os.getenv("AVAILABLE_DEVICES")

# Enable expandable CUDA segments
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# load cuda
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

There are 1 GPU(s) available.
CUDA is available. Using GPU: NVIDIA L40S


In [10]:
# load training variables
FEEDBACK_TO_TRAIN_ON = os.getenv("FEEDBACK_TO_TRAIN_ON")
FEEDBACK_TO_REMOVE = os.getenv("FEEDBACK_TO_REMOVE")
MODEL = os.getenv("REWARD_MODEL")
LORA_CHECKPOINTS_FOLDER = os.getenv("LORA_CHECKPOINTS_FOLDER")
FINAL_LORA_ADAPTERS = os.getenv("FINAL_LORA_ADAPTERS_FOLDER") + "_" + FEEDBACK_TO_TRAIN_ON

# load training data
FILE_1 = os.getenv("FILE_1")
FILE_5 = os.getenv("FILE_5")
FILE_7 = os.getenv("FILE_7")
FILE_9 = os.getenv("FILE_9")
FILE_10_1 = os.getenv("FILE_10_1")
FILE_10_2 = os.getenv("FILE_10_2")
FILE_SYNTH = os.getenv("FILE_SYNTH")

MAX_LENGTH = 512
STRIDE = 128

## 2. Dataset loading and preprocessing

In [11]:
# load dataframes
df_1 = pd.read_csv(FILE_1, sep=";")
df_5 = pd.read_csv(FILE_5, sep=";")
df_7 = pd.read_csv(FILE_7, sep=";")
df_9 = pd.read_csv(FILE_9, sep=";")
df_10_1 = pd.read_csv(FILE_10_1, sep=";")
df_10_2 = pd.read_csv(FILE_10_2, sep=";")
df_synth = pd.read_csv(FILE_SYNTH, sep=";")

df_human = pd.concat([df_1, df_5, df_7, df_9, df_10_1, df_10_2], ignore_index=True)

In [12]:
df_train = df_human
df_train.shape

(929, 13)

### 2. a) Parse ratings to numeric values for MSE Loss

In [13]:
df_train[FEEDBACK_TO_TRAIN_ON] = [parse_ratings(feedback) for feedback in df_train[FEEDBACK_TO_TRAIN_ON]]
print("Parsed feedback for extraction:", df_train[FEEDBACK_TO_TRAIN_ON][:5])

Parsed feedback for extraction: 0    2
1    2
2    2
3    2
4    3
Name: feedback_extraction, dtype: object


### 2. b) keep only relevant feedback column

In [14]:
dataset = Dataset.from_pandas(df_train)

print(dataset)
print(FEEDBACK_TO_TRAIN_ON) 

Dataset({
    features: ['file', 'frame_ID', 'frame_type', 'frame_text', 'precondition_id', 'precondition_text', 'precondition_position', 'response_text', 'prompt_config_examples', 'prompt_config_chain_of_thought', 'feedback_extraction', 'feedback_detection', 'additional_feedback'],
    num_rows: 929
})
feedback_extraction


In [15]:
dataset = dataset.remove_columns([FEEDBACK_TO_REMOVE])
dataset = dataset.rename_column(FEEDBACK_TO_TRAIN_ON, "label")

## 3. Load model with LoRA layer

In [16]:
# Load the model and the tokenizer
model_id = MODEL 
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=1) # num_labels = 1 since we want to prodict a single scalar (the rating)

# Comment: Automodel for sequence classification with num_labels=1 already has a regression head
print(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
print(tokenizer.model_max_length)
print(model.config.max_position_embeddings)

512
512


In [18]:
# Define LoRA config
lora_config = LoraConfig(
    r=8,           # Rank of the LoRA matrices (smaller = less memory)
    lora_alpha=16, # Scaling factor (higher = stronger adaptation)
    target_modules=["query", "key", "value"], # Apply LoRA to attention layers
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"  # classify each answer 
)

# Convert the model to a PEFT (LoRA) model
model = get_peft_model(model, lora_config)
# model.gradient_checkpointing_enable()
model.print_trainable_parameters()  # Check trainable params (~0.1% of full model)


trainable params: 443,137 || all params: 109,926,146 || trainable%: 0.4031


In [19]:
# Test tokenizer
sample_data = ["What is the capital of France?", "What is the largest capital in the world?"]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1067, 223, 207, 580, 210, 1335, 124, 102, 0, 0, 0], [101, 1067, 223, 207, 5601, 190, 580, 213, 207, 1727, 124, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

## 4. Define Custom Trainer to be used for the task

In [20]:
class CustomRewardTrainer(Trainer):
    def __init__(self, *args, loss_type="mse", weight_strategy=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_type = loss_type  # "mse", "huber", or custom
        self.weight_strategy = weight_strategy  # "linear", "inverse", or None

    def compute_loss(self, model, inputs, num_items_in_batch=None, return_outputs=False):
        # Extract labels (ratings) and optional sample weights
        labels = inputs.pop("labels").float()  # Shape: (batch_size)
        
        # Optional: Compute sample weights dynamically
        weights = self._get_sample_weights(labels) if self.weight_strategy else None
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits.squeeze()  # Shape: (batch_size) --> logits are the predicted rewards in this case
        
        # Custom loss calculation
        loss = self._compute_custom_loss(logits, labels, weights)
        
        return (loss, outputs) if return_outputs else loss

    def _compute_custom_loss(self, logits, labels, weights=None):
        if self.loss_type == "mse":
            loss = F.mse_loss(logits, labels, reduction="none") # --> MSE provides precise regression BUT sensitive to outliers
        elif self.loss_type == "huber":
            loss = F.huber_loss(logits, labels, reduction="none", delta=1.0) #--> balances between MSE and MAE for data that has outliers/ noise
        else:
            raise ValueError(f"Unsupported loss type: {self.loss_type}")

        # Apply sample weights if provided
        if weights is not None:
            loss = loss * weights
            loss = loss.mean()  # Normalize by mean if weights are unnormalized
        else:
            loss = loss.mean()
        
        return loss

    def _get_sample_weights(self, labels):
        """
        Generate sample weights based on rating values.
        
        
        """
        if self.weight_strategy == "linear":
            # Linear weighting (e.g., emphasize extremes)
            weights = torch.abs(labels - labels.mean()) + 1.0
        elif self.weight_strategy == "inverse":
            # Inverse frequency weighting (if ratings are skewed)
            unique, counts = torch.unique(labels, return_counts=True)
            freq = counts.float() / len(labels)
            weight_map = 1.0 / (freq + 1e-6)  # Avoid division by zero
            weights = torch.tensor([weight_map[(unique == lbl).nonzero().item()] for lbl in labels])
        else:
            weights = None
        
        return weights.to(labels.device) if weights is not None else None



    def compute_metrics(self, eval_preds):
        predictions, labels = eval_preds
        predictions = predictions.squeeze()
        
        # Regression metrics
        mse = mean_squared_error(labels, predictions)
        pearson = pearsonr(labels, predictions)[0] # Pearson correlation coefficient
        
        # Threshold accuracy --> 
        tolerance_acc = (np.abs(predictions - labels) <= 0.5).mean()
        
        return {"mse": mse, "pearson": pearson, "tolerance_acc": tolerance_acc}
    


    
    #TODO: evaluate whether the plotting should be done or whether it is redundant to add them

    # def evaluation_loop(self, *args, **kwargs):
    #     output = super().evaluation_loop(*args, **kwargs)
    #     predictions = output.predictions.squeeze()
    #     labels = output.label_ids
        
    #     # Generate plots (saved to disk or logged to W&B)
    #     plot_distributions(predictions, labels, self.state.epoch)
    #     plot_calibration(predictions, labels)
        
    #     return output

In [21]:
#TODO: debug if this is truly needed...

# add distributioncallback to trianing to evaluate 

# class DistributionCallback(TrainerCallback):
#     def on_evaluate(self, args, state, control, metrics, **kwargs):
#         # Get predictions and labels from the trainer's eval loop
#         eval_results = trainer.evaluate()
#         predictions = eval_results["eval_predictions"]
#         labels = eval_results["eval_labels"]
        
#         # Log histogram to W&B
#         wandb.log({
#             "reward_histogram": wandb.Histogram(predictions),
#             "true_ratings_histogram": wandb.Histogram(labels),
#         })


# def plot_distributions(predictions, labels, epoch):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.hist(predictions, bins=20, alpha=0.7, label="Predicted")
#     plt.title("Predicted Rewards")
    
#     plt.subplot(1, 2, 2)
#     plt.hist(labels, bins=20, alpha=0.7, label="True Ratings", color="orange")
#     plt.title("True Ratings")
    
#     plt.savefig(f"distributions_epoch_{epoch}.png")
#     plt.close()

# class PlotCallback(TrainerCallback):
#     def on_evaluate(self, args, state, control, **kwargs):
#         predictions = trainer.predict(test_dataset).predictions.squeeze()
#         labels = test_dataset["ratings"]
#         plot_distributions(predictions, labels, state.epoch)




# def plot_calibration(predictions, labels):
#     """
#     Function to check if the sdfs

    

#     """
#     bin_means = np.linspace(1, 5, num=5)  # For 1-5 ratings
#     bin_centers = []
#     empirical_means = []
    
#     for i in range(len(bin_means) - 1):
#         mask = (labels >= bin_means[i]) & (labels < bin_means[i+1])
#         if mask.sum() > 0:
#             bin_centers.append((bin_means[i] + bin_means[i+1]) / 2)
#             empirical_means.append(predictions[mask].mean())
    
#     plt.plot(bin_centers, empirical_means, marker="o")
#     plt.plot([1, 5], [1, 5], linestyle="--", color="gray")  # Ideal line
#     plt.xlabel("True Rating")
#     plt.ylabel("Predicted Reward")
#     plt.savefig("calibration_plot.png")



### 4. a) Define a custom data collator

In [22]:
# from transformers import DefaultDataCollator

# #TODO: only do this if the labels fix does not work for some reason

# class RewardDataCollator(DefaultDataCollator):
#     def __call__(self, features):

#         ratings = [f.pop("rating") for f in features]  # Removes rating from features temporarily
#         batch = super().__call__(features)
#         # Explicitly ensure rating is included
#         print(features)
#         # Re-inject ratings into the batch
#         batch["rating"] = torch.tensor(ratings, dtype=torch.float32)
#         return batch

## 5. Encode dataset

In [23]:
# if labels are not integers, convert them to integers
def convert_label_to_int(data):
    data["label"] = int(data["label"])
    return data


print(dataset.column_names)
# mao string labels to integers
dataset = dataset.map(convert_label_to_int)  # Assuming 'text' is the column with the text data

print(dataset["label"][:5])  # Check labels
print(dataset["response_text"][:5])  # Check labels

['file', 'frame_ID', 'frame_type', 'frame_text', 'precondition_id', 'precondition_text', 'precondition_position', 'response_text', 'prompt_config_examples', 'prompt_config_chain_of_thought', 'label', 'additional_feedback']


Map: 100%|██████████| 929/929 [00:00<00:00, 4253.68 examples/s]

[2, 2, 2, 2, 3]
['1. Subfact: Onze Minister\n                2. Positie: Artikel 1, sectie 1 IN Vreemdelingenwet geldig vanaf 2024\n                3. Subfact: Onze Minister\n                4. Positie: Artikel 8, sectie 1 IN Vreemdelingenwet geldig vanaf 2024\n                5. Subfact: Onze Minister\n                6. Positie: Artikel 14, sectie 1 IN Vreemdelingenwet geldig vanaf 2024\n                7. Subfact: Onze Minister\n                8. Positie: Artikel 16, sectie 1 IN Vreemdelingenwet geldig vanaf 2024\n                9. Subfact: Onze Minister\n                10. Positie: Artikel 17, sectie 1 IN Vreemdelingenwet geldig vanaf 2024\n                11. Subfact: Onze Minister\n                12. Positie: Artikel 17a, sectie 1 IN Vreemdelingenwet geldig vanaf 2024\n                13. Subfact: Onze Minister\n                14. Positie: Artikel 26, sectie 1 IN Vreemdelingenwet geldig vanaf 2024\n                15. Subfact: Onze Minister\n                16. Positie: Arti

## Comment

1. Needed for feedback extraction: precondition_text, response_text, label(rating feedback extraction)
2. Needed for feedback detection: precondition_text, precondition_position, response_text, label (rating feedback detection)
3. For the precondition position to be found well, it is a crucial for the model to find the precondition text (at least to a recognizable degree) as well, otherwise the precondition is not found at all...

In [24]:
# from sklearn.feature_extraction.text import TfidfVectorizer


def find_best_window(long_text, ground_truth, tokenizer, window_size=512, stride=256):
    """
    Find the best sliding window of text that is most similar to the ground truth.
    Uses a semantic similarity model to evaluate the windows.
    """
    similarity_model = SentenceTransformer('NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers')
    similarity_model.to(device)
    
    # Tokenize the full text
    tokens = tokenizer.tokenize(long_text)
    
    best_score = -1
    best_window = None

    
    # Handle special cases in sequence length
    if len(tokens) < window_size:
        # Handle short sequences (optional: pad or skip)
        start_indices = [0]
    elif len(tokens) - window_size + 1 < stride:
        # Not enough room for a second stride step
        start_indices = [0, len(tokens) - window_size]
    else:
        # Standard sliding window
        start_indices = list(range(0, len(tokens) - window_size + 1, stride))

    
    # Create sliding windows
    for start in start_indices:
        window_tokens = tokens[start:start + window_size]
        window_text = tokenizer.convert_tokens_to_string(window_tokens)
        
        # Calculate cosine similarity with ground truth
        embeddings = similarity_model.encode([window_text, ground_truth])
        similarity = np.dot(embeddings[0], embeddings[1]) / (
            np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1])
        )
        
        if similarity > best_score:
            best_score = similarity
            best_window = window_tokens

    # print(best_window)
    if best_window == None:
        print(similarity)
        print(ground_truth)
        print(long_text)
    
    return tokenizer.convert_tokens_to_string(best_window)

In [25]:
# tokenize queries and answers together to provide proper context to reward model
def tokenize_fn_with_best_window(examples):

    """
    Tokenization function choosing the best window in the answer using similarity score with ground truth.
    """

    max_length = MAX_LENGTH
    stride = STRIDE

    #TODO: need to replace the tokenize function here, get the number fo tokens properly
    response_tokens = len(tokenizer(examples["response_text"], truncation=False)[0])
    # print(response_tokens)
    precond_tokens = len(tokenizer(examples["precondition_text"], truncation=False)[0])
    # print(precond_tokens)
    pos_tokens = len(tokenizer(examples["precondition_position"], truncation=False)[0])
    # print(pos_tokens)

    precon_text = examples["precondition_text"]
    precon_pos = examples["precondition_position"]
    response = examples["response_text"]

    # combine the text for each separate use case
    if FEEDBACK_TO_TRAIN_ON == "feedback_extraction":
        # print("Tokenizing for feedback extraction")
        # choose the best sequence if the number of tokens is more than 512
        num_tokens = response_tokens + precond_tokens
        if num_tokens > max_length:
            window_size = max_length - precond_tokens
            stride = window_size // 2
            ground_truth = precon_text
            text = find_best_window(response, ground_truth, tokenizer, window_size=window_size, stride=stride)
            # print(text)
            # combined_texts = [f"{t} {r}" for t, r in zip(examples["precondition_text"], text)] --> batched version
            combined_texts = f"{precon_text} {precon_pos} {text}"
        else:
            # combined_texts = [f"{t} {r}" for t, r in zip(examples["precondition_text"], examples["response_text"])] --> batched version
            combined_texts = f"{precon_text} {precon_pos} {response}"
    
    
    
    
    elif FEEDBACK_TO_TRAIN_ON == "feedback_detection":
        # print("Tokenizing for feedback detection")
        num_tokens = response_tokens + precond_tokens + pos_tokens
        # print(num_tokens)
        # print(max_length)
        # print(examples["response_text"])
        if num_tokens > max_length:
            # print("We get here at all")
            window_size = max_length - (precond_tokens + pos_tokens)
            stride = window_size // 2
            ground_truth = precon_text + " " + precon_pos
            text = find_best_window(response, ground_truth, tokenizer, window_size=window_size, stride=stride)

            #combined_texts = [f"{t} {p} {r}" for t, p, r in zip(examples["precondition_text"], examples["precondition_position"], text)] --> batched version
            
            combined_texts = f"{precon_text} {precon_pos} {text}"
        else:
            #combined_texts = [f"{t} {p} {r}" for t, p, r in zip(examples["precondition_text"], examples["precondition_position"], examples["response_text"])] --> batched version
            combined_texts = f"{precon_text} {precon_pos} {response}"

    tokenized = tokenizer(combined_texts, truncation=True, padding="max_length")

    return tokenized



# sample = dataset.select(range(5))
# tokenized_sample = tokenize_fn(sample)
# print([len(ids) for ids in tokenized_sample["input_ids"]])

dataset = dataset.map(tokenize_fn_with_best_window, batched=False)

Map:   4%|▍         | 37/929 [00:25<10:15,  1.45 examples/s]


KeyboardInterrupt: 

In [ ]:
# Assuming `encoded_dataset` is a Hugging Face Dataset object
dataset.save_to_disk(f"/home/jacques.furst/development/RAG/flintfiller-precondition-rl/training_files/tokenized_data_reward_model_{FEEDBACK_TO_TRAIN_ON}")
dataset = load_from_disk(f"/home/jacques.furst/development/RAG/flintfiller-precondition-rl/training_files/tokenized_data_reward_model_{FEEDBACK_TO_TRAIN_ON}")

Saving the dataset (1/1 shards): 100%|██████████| 929/929 [00:00<00:00, 162280.14 examples/s]


# Split dataset into train, test, eval

In [ ]:
# split into train, test and eval sets
train_test_split = dataset.train_test_split(test_size=0.3, seed=42)
eval_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)


final_splits = DatasetDict({
    'train': train_test_split['train'],
    'validation': eval_test_split['train'],
    'test': eval_test_split['test']
})

## 6. Train reward model

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=LORA_CHECKPOINTS_FOLDER,
    eval_strategy='steps',
    save_strategy='steps',
    save_steps=100,
    eval_steps=100,
    save_total_limit=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-4,
    num_train_epochs=20,
    logging_steps=10,
    label_names=["labels"],
    # report_to="none",
    logging_dir="./logs",
    fp16=True,  # Use mixed precision training
    metric_for_best_model="eval_loss", # or "eval_loss"
    greater_is_better=False, # False if using loss
    # gradient_accumulation_steps=4 # 
)

# Initialize custom trainer
trainer = CustomRewardTrainer(
    model=model,
    args=training_args,
    train_dataset=final_splits['train'],
    eval_dataset=final_splits['validation'],
    # compute_metrics=trainer.compute_metrics,  # Use the custom metrics function
    processing_class=tokenizer,
    loss_type="huber",  # "mse" or "huber"
    weight_strategy="linear",  # "linear", "inverse", or None
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # use early stopping since we are sing high amount of epochs
    # data_collator=RewardDataCollator()
)

# # add distributioncallback to trainer TODO: only integrate if relevant
# trainer.add_callback(DistributionCallback())

print(trainer.args.device)

cuda:0


In [ ]:
#train model
trainer.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss
100,0.488800,0.435845
200,0.414100,0.618522
300,0.358900,0.293196
400,0.178100,0.272691
500,0.267500,0.264794
600,0.204800,0.285815
700,0.230100,0.289056
800,0.282800,0.285740


TrainOutput(global_step=800, training_loss=0.5441169528663159, metrics={'train_runtime': 60.1869, 'train_samples_per_second': 215.994, 'train_steps_per_second': 13.624, 'total_flos': 3355066545033216.0, 'train_loss': 0.5441169528663159, 'epoch': 19.51219512195122})

In [ ]:
# store final model parameters
model.save_pretrained(FINAL_LORA_ADAPTERS)

# Reload saved LoRA adapter for inference 

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=1)
model = PeftModel.from_pretrained(base_model, FINAL_LORA_ADAPTERS)
model = model.merge_and_unload()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=final_splits['test'])
print("Test Results:", test_results)

Test Results: {'eval_loss': 0.40426239371299744, 'eval_runtime': 0.2921, 'eval_samples_per_second': 479.328, 'eval_steps_per_second': 30.814, 'epoch': 19.51219512195122}


In [ ]:
# evaluate model manually on some test cases
model.to(device)
model.eval()

with torch.no_grad():
    for i in range(20):
        sample = final_splits['test'][i]
        inputs = tokenizer(sample['precondition_text'] + " " + sample['response_text'], return_tensors='pt', truncation=True, padding="max_length").to(device)
        outputs = model(**inputs)
        prediction = outputs.logits.item()
        print(f"Sample {i+1}: Predicted Rating: {prediction}, True Rating: {sample['label']}")


Sample 1: Predicted Rating: 4.385091781616211, True Rating: 6
Sample 2: Predicted Rating: 6.064860820770264, True Rating: 6
Sample 3: Predicted Rating: 5.961627960205078, True Rating: 4
Sample 4: Predicted Rating: 6.023962497711182, True Rating: 6
Sample 5: Predicted Rating: 4.425707817077637, True Rating: 4
Sample 6: Predicted Rating: 6.039861679077148, True Rating: 6
Sample 7: Predicted Rating: 4.0829291343688965, True Rating: 4
Sample 8: Predicted Rating: 6.074625015258789, True Rating: 6
Sample 9: Predicted Rating: 6.0770649909973145, True Rating: 6
Sample 10: Predicted Rating: 6.0072479248046875, True Rating: 6
Sample 11: Predicted Rating: 5.0286335945129395, True Rating: 6
Sample 12: Predicted Rating: 5.976753234863281, True Rating: 6
Sample 13: Predicted Rating: 6.047481536865234, True Rating: 6
Sample 14: Predicted Rating: 6.014069557189941, True Rating: 6
Sample 15: Predicted Rating: 4.109388828277588, True Rating: 6
Sample 16: Predicted Rating: 5.910253047943115, True Rating:

In [ ]:
print(FEEDBACK_TO_TRAIN_ON) 

feedback_detection
